# Keras example with h5py model saving
---

<font color='red'> <h3>Tested with TensorFlow 1.10</h3></font>
<font color='red'> <h3>This notebook requires h5py pip library, please install it in Hopsworks.</h3></font>

## Hops Experiment paradigm <a class="anchor" id='paradigm'></a>

To be able to run your Keras code on Hops, the code for the whole program needs to be provided and put inside a wrapper function. Everything, from importing libraries to reading data and defining the model and running the program needs to be put inside a wrapper function. If you wish to run gridsearch over a given set of hyperparameters, you can define arguments for this wrapper function that corresponds to the name of your hyperparameters.

You can also submit one or more `.py`, `.zip` or `.egg` files that contain your code and import them in the wrapper function. To include files, navigate back to HopsWorks and restart restart Jupyter, you can then include files in the Jupyter configuration.

## The `hops` python module

Below you can see the aforementioned wrapper function, which is coincidently named `wrapper` but could potentially be named anything. You can see two imports from the `hops` module, a `tensorboard` and an `hdfs` module. These are the only two modules that you will need to use in your Keras wrapper function. 

### Using the `tensorboard` module
The `tensorboard` module allow us to get the log directory for summaries and checkpoints to be written to the TensorBoard we will see in a bit. The only function that we currently need to call is `tensorboard.logdir()`, which returns the path to the TensorBoard log directory. Furthermore, the content of this directory will be put in as a Dataset in your project in HopsFS after each hyperparameter configuration is finished. The `experiment.launch` function, that we will look at abit further down will return the exact path, which you can then navigate to using HopsWorks to inspect the files.

The directory could in practice be used to store other data that should be accessible after each hyperparameter configuration is finished.
```python
# Use this module to get the TensorBoard logdir
from hops import tensorboard
tensorboard_logdir = tensorboard.logdir()
```


### Using the `hdfs` module
The `hdfs` module provides a single method to get the path in HopsFS where your data is stored, namely by calling `hdfs.project_path()`. The path resolves to the root path for your project, which is the view that you see when you click `Data Sets` in HopsWorks. To point where your actual data resides in the project you to append the full path from there to your Dataset. For example if you create a mnist folder in your Resources Dataset, which is created automatically for each project, the path to the mnist data would be `hdfs.project_path() + 'Resources/mnist'`
```python
# Use this module to get the path to your project in HopsFS, then append the path to your Dataset in your project
from hops import hdfs
project_path = hdfs.project_path()
```

![image11-Dataset-ProjectPath.png](../../images/datasets.png)

In [ ]:
def keras_mnist():
    from tensorflow.python import keras
    import tensorflow as tf
    from tensorflow.python.keras.datasets import mnist
    from tensorflow.python.keras.models import Sequential
    from tensorflow.python.keras.layers import Dense, Dropout, Flatten
    from tensorflow.python.keras.layers import Conv2D, MaxPooling2D
    from tensorflow.python.keras.callbacks import TensorBoard
    from tensorflow.python.keras import backend as K

    import math
    from hops import tensorboard

    batch_size = 128
    num_classes = 10
    epochs = 1
    kernel = 4
    pool = 4
    dropout = 0.5

    # Input image dimensions
    img_rows, img_cols = 28, 28

    # The data, shuffled and split between train and test sets
    (x_train, y_train), (x_test, y_test) = mnist.load_data()

    if K.image_data_format() == 'channels_first':
        x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
        x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
        input_shape = (1, img_rows, img_cols)
    else:
        x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
        x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
        input_shape = (img_rows, img_cols, 1)

    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # Convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)

    model = Sequential()
    model.add(Conv2D(32, kernel_size=(kernel, kernel),
                        activation='relu',
                         input_shape=input_shape))
    model.add(Conv2D(64, (kernel, kernel), activation='relu'))
    model.add(MaxPooling2D(pool_size=(pool, pool)))
    model.add(Dropout(dropout))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(num_classes, activation='softmax'))

    opt = keras.optimizers.Adadelta(1.0)

    model.compile(loss=keras.losses.categorical_crossentropy,
                      optimizer=opt,
                      metrics=['accuracy'])

    tb_callback = TensorBoard(log_dir=tensorboard.logdir(), histogram_freq=0,
                             write_graph=True, write_images=True)
    callbacks = [tb_callback]
    callbacks.append(keras.callbacks.ModelCheckpoint(tensorboard.logdir() + '/checkpoint-{epoch}.h5'))

    model.fit(x_train, y_train,
             batch_size=batch_size,
             callbacks=callbacks,
             epochs=epochs,
             verbose=1,
             validation_data=(x_test, y_test))
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])

In [ ]:
from hops import experiment
from hops import hdfs

notebook = hdfs.project_path() + '/Jupyter/Experiment/Keras/mnist.ipynb'
experiment.launch(keras_mnist, name='keras mnist', local_logdir=True)

## Monitoring execution - TensorBoard <a class="anchor" id='tensorboard'></a>
To find the TensorBoard for the execution, please go back to HopsWorks and enter the Experiments service.
Then copy & paste the experiment_id into the textbox and press enter to start a TensorBoard to see all experiments being run in parallel.

![Image7-Monitor.png](../../images/experiments_service.png)
![Image7-Monitor.png](../../images/tensorboard.png)